In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
#from pytorch3d.loss import chamfer_distance
import importlib

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import open3d as o3d
import os
import torch
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import sys
from pathlib import Path
import time
#from pytorch3d.loss import chamfer_distance

sys.path.append(str(Path.cwd().parent))
from Helpers.data import PointCloudDataset
from Helpers.repair_dataset import repair_off_files
import Helpers.PointCloudOpen3d as pc

if torch.cuda.is_available():
    device = "cuda"

elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using: cuda


In [9]:
dataset_dir = "../Data/ModelNet40"
repair_off_files(dataset_dir)

In [37]:
point_size = 1024
object_classes= ['chair']#['stool', 'chair', 'tv_stand', 'bench', 'desk', 'sofa', 'wardrobe', 'toilet', 'table', 'sink', 'night_stand', 'mantel','dresser', 'bookshelf','bed']
train_dataset = PointCloudDataset("../Data/ModelNet40", point_size, 'train', object_classes=object_classes)
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
# test_dataset = PointCloudDataset("../Data/ModelNet40", point_size, 'test', object_classes=object_classes)
# test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)
# print(len(test_dataset))
print(len(train_dataset))


889


426


In [6]:
class PointCloudAE(nn.Module):
    def __init__(self, point_size, latent_size):
        super(PointCloudAE, self).__init__()
        
        self.latent_size = latent_size
        self.point_size = point_size
        
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, self.latent_size, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(self.latent_size)
        
        self.dec1 = nn.Linear(self.latent_size,256)
        self.dec2 = nn.Linear(256,256)
        self.dec3 = nn.Linear(256,self.point_size*3)

    def encoder(self, x): 
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, self.latent_size)
        return x
    
    def decoder(self, x):
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = self.dec3(x)
        return x.view(-1, self.point_size, 3)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [39]:
def chamfer_distance(x, y):
    """
    Compute Chamfer Distance between two point clouds x and y.
    x: (batch_size, n_points, 3)
    y: (batch_size, m_points, 3)
    Returns: Chamfer distance loss (a scalar)
    """
    # Compute pairwise Euclidean distance between points in x and y
    dist = torch.cdist(x, y, p=2)  # dist: (batch_size, n_points, m_points)
    
    # Find the minimum distance from each point in x to y
    dist1, _ = torch.min(dist, dim=2)  # (batch_size, n_points), min distances for x to y
    
    # Find the minimum distance from each point in y to x
    dist2, _ = torch.min(dist, dim=1)  # (batch_size, m_points), min distances for y to x
    
    # Chamfer Distance = average of both directions
    chamfer_loss = torch.mean(dist1) + torch.mean(dist2)
    return chamfer_loss

In [40]:

net = PointCloudAE(1024, 256).to(device)

optimizer = optim.AdamW(net.parameters(), lr=0.0001)

def train_epoch():
    epoch_loss = 0
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        
        x = data['points'].to(device)

        output = net(x.permute(0,2,1)) # transpose data for NumberxChannelxSize format

        loss = chamfer_distance(x, output)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    if i == 0:
        return epoch_loss / 1
    return epoch_loss/i


train_loss_list = []  
test_loss_list = []  

for i in range(10000) :

    startTime = time.time()
    
    train_loss = train_epoch() #train one epoch, get the average loss
    train_loss_list.append(train_loss)

    
    epoch_time = time.time() - startTime
    
    if i % 10 == 0:
        print(f"Epoch: {i} Loss: {train_loss}")

Epoch: 0 Loss: 0.6768216215647184
Epoch: 10 Loss: 0.13773161860612723
Epoch: 20 Loss: 0.10007947511397876
Epoch: 30 Loss: 0.09124401383675061
Epoch: 40 Loss: 0.08557815677844562
Epoch: 50 Loss: 0.08477511428869687
Epoch: 60 Loss: 0.08347640931606293
Epoch: 70 Loss: 0.08144808961794926
Epoch: 80 Loss: 0.07883809334956683
Epoch: 90 Loss: 0.08115584861773711
Epoch: 100 Loss: 0.07646366667289001
Epoch: 110 Loss: 0.07833686241736779
Epoch: 120 Loss: 0.07460961605493839
Epoch: 130 Loss: 0.07318866023650536
Epoch: 140 Loss: 0.07365244512374584
Epoch: 150 Loss: 0.07100835729103822
Epoch: 160 Loss: 0.07228594789138207
Epoch: 170 Loss: 0.0715901109461601
Epoch: 180 Loss: 0.07256328113950215
Epoch: 190 Loss: 0.06879754994924252
Epoch: 200 Loss: 0.06993510917975353
Epoch: 210 Loss: 0.0686522197838013
Epoch: 220 Loss: 0.06909783786306015
Epoch: 230 Loss: 0.06866265890689996
Epoch: 240 Loss: 0.06885016738222195
Epoch: 250 Loss: 0.06565688607784417
Epoch: 260 Loss: 0.06836459843012002
Epoch: 270 Loss

In [27]:
torch.save(net.state_dict(), 'trained_models/furniture_10k')

In [49]:
torch.save(net, 'trained_models/furniture_model_weights_50k')

In [50]:
model = torch.load('trained_models/furniture_model_weights_50k', weights_only = False)

In [36]:
model = net
x = next(iter(train_loader))['points'][0]

cloud = pc.get_point_cloud(x[:1000])
pc.visualize_point_cloud(cloud)
cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)

with torch.no_grad():
    data = x.unsqueeze(0).permute(0,2,1).to(device)

    rec_x = np.array(model(data)[0].to('cpu'))
    cloud = pc.get_point_cloud(rec_x)
    pc.visualize_point_cloud(cloud)
# x = next(iter(train_loader))['points'][0]

# cloud = pc.get_point_cloud(x)
# pc.visualize_point_cloud(cloud)

# with torch.no_grad():
#     data = x.unsqueeze(0).permute(0,2,1).to(device)

#     rec_x = np.array(model(data)[0].to('cpu'))
#     cloud = pc.get_point_cloud(rec_x)
#     pc.visualize_point_cloud(cloud)

In [52]:
model = net
x = next(iter(test_loader))['points'][0]

cloud = pc.get_point_cloud(x)
pc.visualize_point_cloud(cloud)

with torch.no_grad():
    data = x.unsqueeze(0).permute(0,2,1).to(device)

    rec_x = np.array(model(data)[0].to('cpu'))
    cloud = pc.get_point_cloud(rec_x)
    pc.visualize_point_cloud(cloud)